#                        Здесь мы будем классифицировать отзывы по типам продуктов. 
                    Для начала импортируем необходимые библиотеки и загружаем обучающую выбурку.

In [1]:
#импортируем необходимые библиотеки
import pandas
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.svm import LinearSVC
#загружаем обучающую выборку
data = pandas.read_csv('train_data.csv',index_col='Unnamed: 0')
data.head()

,type,title,text
0,Кредит,Просьба об отмене штрафа КАСКО,Здравствуйте! 10.06.2016г. мною был заключен д...
1,Ипотека,"В целом хорошо, работали по ипотеке.","Брала ипотеку в юникоре, порадовали квалифика..."
2,Реструктуризация,Нас всегда выручает банк Уралсиб!,Знакомство с банком началось с ипотеки в 2009 ...
3,Вклад,Абсолютно неприемлемая ситуация и ее решение,Клиент Совкомбанка с 2017 года. В феврале 2018...
4,Кредит,Внёс не туда,Пришёл оплачивать кредит и ошибочно внёс деньг...


                        Смотрим на уникальные значения категорий.

In [2]:
#смотрим какие значения может принимать type
pandas.unique((data['type'].values).tolist())

array(['Кредит', 'Ипотека', 'Реструктуризация', 'Вклад', 'Бизнес услуги',
       'Обслуживание физ. и юр. лиц', 'Дебетовая карта',
       'Денежные переводы', 'Инвестиционные продукты'], dtype=object)

In [3]:
#заменяем строковые переменные на целые значения для классификации
data['type'] = data['type'].map({'Кредит': 0, 'Ипотека': 1,'Реструктуризация': 2,'Вклад':3,
                                 'Бизнес услуги': 4,'Обслуживание физ. и юр. лиц': 5, 'Дебетовая карта':6,
                                 'Денежные переводы':7,'Инвестиционные продукты':8})

In [4]:
#сохраняем значения type в целевую переменную
y = data['type'].values

                Теперь обрабатываем сам текст и приводим его к нужному для дальнейшей работы виду.

In [ ]:
#объединяем заголовок с отзывом в один текст для анализа и приводим к нижнему регистру
for index,row in data.iterrows():
    data['text'][index]=row['title']+" "+row['text']
for index,row in data.iterrows():    
    data['text'][index]=row['text'].lower()

In [ ]:
#очищаем текст от знаков припинания и чисел 
reg = re.compile('[^а-яА-я ]')
for index,row in data.iterrows():
    data['text'][index]=reg.sub('',row['text'])

In [7]:
#удаляем ненужные столбцы
data=data.drop(['type','title'],1)

In [8]:
#составляет словарь стоп-слов
dictionary=["и","в","на","с","мне","я","банк","для", "по", "со", "из", "от", "до", "без", "над", "под", "за", "при", "после", 
            "во","это","или", "но", "дабы", "затем", "потом", "коли", "лишь только","как", "так", "еще", "тот", "откуда", "зачем",
            "почему", "значительно","он", "мы", "его", "вы", "вам", "вас", "ее", "что", "который","которая","которое", "их", 
            "все", "они", "я", "весь", "мне", "меня", "таким", "весь", "всех","кб", "мб", "дн", "руб", "ул", "кв", "дн", "гг",
            "всего", "почти", "примерно", "около", "порядка","скажем", "может", "допустим", "например", 
            "однако", "вообще", "вообщем", "вероятно", "скорее","ой", "ого", "эх", "браво", 
            "здравствуйте", "спасибо", "извините", "пожалуйста","банком","банке","банка","не","к"]

In [ ]:
#очищаем данные от стоп-слов и применяем TF-IDF
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.7,analyzer='word',stop_words=dictionary)
data_vectorized = vectorizer.fit_transform(data['text'])

            В этом блоке выбираем наиболее информативные признаки и смотрим на показатель качества лучшей модели.

In [10]:
#выбираем наиболее информативные признаки
ch2 = SelectKBest(chi2, k=25000)
X_train = ch2.fit_transform(data_vectorized, y)

In [11]:
#проверяем качество нашей лучшей модели на кросс-валидации
kf = model_selection.KFold(n_splits=5,shuffle=True,random_state=22)
lsvc=LinearSVC(C=0.8,tol=0.8,random_state=22)
sc=model_selection.cross_val_score(lsvc, X_train, y,cv=kf,scoring='accuracy')
print(f"Score:{sc.mean()}")

Score:0.7028737618369728


                        Здесь готовим тестовую выборку к обработке и получаем наш ответ.

In [ ]:
#загружаем тестовую выборку
test_data=pandas.read_csv('test_data.csv')
#преобразуем тестовую выборку 
for index,row in test_data.iterrows():
    test_data['text'][index]=row['title']+" "+row['text']
for index,row in test_data.iterrows():    
    test_data['text'][index]=row['text'].lower()
for index,row in test_data.iterrows():
    test_data['text'][index]=reg.sub('',row['text'])    

test_data=test_data.drop(['Unnamed: 0','title'],1)
test_data_vectorized = vectorizer.transform(test_data['text'])
X_test = ch2.transform(test_data_vectorized)
#задаем и обучаем классификатор
lsvc=LinearSVC(C=0.8,tol=0.8,random_state=22)
lsvc.fit(X_train, y)
#делаем предсказания
pred = lsvc.predict(X_test)
#готовим предсказания к переводу в csv файл
ind=[]
for i in range(0,2750):
    ind.append(i)
types=[]
for i in pred:
    if i==0:
        types.append('Кредит')
    if i==1:
        types.append('Ипотека')
    if i==2:
        types.append('Реструктуризация')
    if i==3:
        types.append("Вклад")    
    if i==4:
        types.append('Бизнес услуги')
    if i==5:
        types.append('Обслуживание физ. и юр. лиц')
    if i==6:
        types.append('Дебетовая карта')
    if i==7:
        types.append('Денежные переводы')
    if i==8:
        types.append('Инвестиционные продукты')  
df_ind=pandas.DataFrame(ind,index=None,columns=['index'])
df_types=pandas.DataFrame(types,index=None,columns=['type'])
df=df_ind.join(df_types)
df.to_csv('class_answer.csv',index=False)